In [1]:
#les imports nécessaires 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# The Antirectifier layer

In [2]:
class Antirectifier(layers.Layer):
    def __init__(self, initializer="he_normal", **kwargs):
        super(Antirectifier, self).__init__(**kwargs)
        self.initializer = keras.initializers.get(initializer)

    def build(self, input_shape):
        output_dim = input_shape[-1]
        self.kernel = self.add_weight(
            shape=(output_dim * 2, output_dim),
            initializer=self.initializer,
            name="kernel",
            trainable=True,
        )

    def call(self, inputs):
        inputs -= tf.reduce_mean(inputs, axis=-1, keepdims=True)
        pos = tf.nn.relu(inputs)
        neg = tf.nn.relu(-inputs)
        concatenated = tf.concat([pos, neg], axis=-1)
        mixed = tf.matmul(concatenated, self.kernel)
        return mixed

    def get_config(self):
        # Implement get_config to enable serialization. This is optional.
        base_config = super(Antirectifier, self).get_config()
        config = {"initializer": keras.initializers.serialize(self.initializer)}
        return dict(list(base_config.items()) + list(config.items()))

# Let's test-drive it on MNIST

In [3]:
# Training parameters
batch_size = 128
num_classes = 10
epochs = 20

# The data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# Build the model
model = keras.Sequential(
    [
        keras.Input(shape=(784,)),
        layers.Dense(256),
        Antirectifier(),
        layers.Dense(256),
        Antirectifier(),
        layers.Dropout(0.5),
        layers.Dense(10),
    ]
)

# Compile the model
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train the model
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.15)

# Test the model
model.evaluate(x_test, y_test)

60000 train samples
10000 test samples
Epoch 1/20
399/399 [==============================] - 9s 17ms/step - loss: 0.3845 - sparse_categorical_accuracy: 0.8860 - val_loss: 0.2063 - val_sparse_categorical_accuracy: 0.9401
Epoch 2/20
399/399 [==============================] - 6s 16ms/step - loss: 0.1779 - sparse_categorical_accuracy: 0.9513 - val_loss: 0.1594 - val_sparse_categorical_accuracy: 0.9582
Epoch 3/20
399/399 [==============================] - 6s 16ms/step - loss: 0.1379 - sparse_categorical_accuracy: 0.9630 - val_loss: 0.1095 - val_sparse_categorical_accuracy: 0.9714
Epoch 4/20
399/399 [==============================] - 7s 17ms/step - loss: 0.1111 - sparse_categorical_accuracy: 0.9692 - val_loss: 0.1406 - val_sparse_categorical_accuracy: 0.9712
Epoch 5/20
399/399 [==============================] - 7s 16ms/step - loss: 0.1001 - sparse_categorical_accuracy: 0.9741 - val_loss: 0.1488 - val_sparse_categorical_accuracy: 0.9664
Epoch 6/20
399/399 [==============================] - 6s

[0.20975200831890106, 0.978600025177002]